In [9]:
import spacy
import pandas as pd
import numpy as np
import json

In [3]:
df = pd.read_csv('../data/full_dataset.csv')

In [4]:
expanded_spacy = spacy.load(r"../models/on_expanded/spacy_fine_tuned")
expanded_bert = spacy.load(r"../models/on_expanded/bert_fine_tuned") #load the best model
original_spacy = spacy.load(r"../models/on_original/spacy_fine_tuned")
original_bert = spacy.load(r"../models/on_original/bert_fine_tuned") #load the best model

/Users/qiwenzhang/opt/anaconda3/lib/python3.8/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.2.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
ex = np.random.choice(df['resume'])

In [6]:
doc = expanded_bert(ex)
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

/Users/qiwenzhang/opt/anaconda3/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [7]:
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                    entities.append((point['start'], point['end'] + 1 ,label))
            training_data.append((text, {"entities": entities}))
        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None

In [10]:
testing_github_data = convert_dataturks_to_spacy('../data/testdata.json')

In [18]:
ex = testing_github_data[2][0]
doc = expanded_bert(ex)
spacy.displacy.render(doc, style="ent", jupyter=True)

In [19]:
ex = testing_github_data[2][0]
doc = original_bert(ex)
spacy.displacy.render(doc, style="ent", jupyter=True)